In [79]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy



if (not os.path.isfile('data/spy-train.csv')):
    start = datetime.datetime(2015, 2, 1)
    end = datetime.datetime.now()
    df_train = data.DataReader("SPY", 'yahoo', start, end)
    df_train.to_csv("data/spy-train.csv")

df_train = pd.read_csv('data/spy-train.csv', index_col='Date', parse_dates=True)
latestDate = datetime.datetime.now() - datetime.timedelta(days=1)
if (df_train.index[-1] < latestDate):
    start = df_train.index[-1] + datetime.timedelta(days=1)
    new_data = data.DataReader("SPY", 'yahoo', start, latestDate)
    df_train.append(new_data)

df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)

df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)
#print(df_train)

In [88]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean()
df_train = df_train.assign(pma10=pma10)

pma20 = df_train['Close'].rolling(window=20,center=False).mean()
df_train = df_train.assign(pma20=pma20)

pma50 = df_train['Close'].rolling(window=50,center=False).mean()
df_train = df_train.assign(pma50=pma50)

pma200 = df_train['Close'].rolling(window=200,center=False).mean()
df_train = df_train.assign(pma200=pma200)

In [103]:
hh = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
df_train = df_train.assign(hh=hh)

hl = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 
df_train = df_train.assign(hl=hl)

In [104]:
print(df_train)

                  Open        High         Low       Close     Volume  \
Date                                                                    
2015-02-02  200.050003  202.029999  197.860001  201.919998  163107000   
2015-02-03  203.000000  204.850006  202.550003  204.839996  124212900   
2015-02-04  203.919998  205.380005  203.509995  204.059998  134306700   
2015-02-05  204.860001  206.300003  204.770004  206.119995   97953200   
2015-02-06  206.559998  207.240005  204.919998  205.550003  125672000   
2015-02-09  204.770004  205.639999  204.139999  204.630005   87219000   
2015-02-10  205.880005  207.119995  204.679993  206.809998   96164200   
2015-02-11  206.610001  207.449997  205.830002  206.929993   91087800   
2015-02-12  207.889999  208.990005  206.970001  208.919998   97545900   
2015-02-13  209.070007  209.839996  208.759995  209.779999   93670400   
2015-02-17  209.399994  210.320007  209.100006  210.110001   76968200   
2015-02-18  209.660004  210.220001  209.339996  210